<a href="https://colab.research.google.com/github/JacopoMangiavacchi/Swift-TensorFlow-Sample-Notebooks/blob/master/LinearRegression_MultiVariable_With_Packages_Swift_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Swift/Python/Pandas** code to download the training/test datasat as Swift arrays

In [0]:
import Python

func getArrayDataset() -> ([[Float]], [Float]) {
    let np = Python.import("numpy")
    let pd = Python.import("pandas")
    let io = Python.import("io")
    let requests = Python.import("requests")

    let url="https://raw.githubusercontent.com/Dataweekends/zero_to_deep_learning_video/master/data/weight-height.csv"
    let s = requests.get(url).content
    let df = pd.read_csv(io.StringIO(s.decode("utf-8")))

    let dummies = pd.get_dummies(df[["Gender"]])
    let trasnformed = pd.concat([df[["Height", "Weight"]], dummies], 1)
    // print(trasnformed)

    let X1 = trasnformed[["Height"]].values
    let X2 = trasnformed[["Gender_Female"]].values
    let X3 = trasnformed[["Gender_Male"]].values
    let Y = trasnformed[["Weight"]].values
  
    let x1Array = X1.tolist().flatMap{ $0.map{ Float($0)! }}
    let x2Array = X2.tolist().flatMap{ $0.map{ Float($0)! }}
    let x3Array = X3.tolist().flatMap{ $0.map{ Float($0)! }}
    let yArray = Y.tolist().flatMap{ $0.map{ Float($0)! }}
  
    return ([x1Array, x2Array, x3Array], yArray)
}

**Swift** function for shell execute

In [0]:
import Foundation

@discardableResult
func shell(_ args: String...) -> Int32 {
    let task = Process()
    task.launchPath = "/usr/bin/env"
    task.arguments = args
    task.launch()
    task.waitUntilExit()
    return task.terminationStatus
}

**Swift** code for importing the Swift Package source code

In [3]:
shell("rm", "-rf", "SwiftNormalization")
shell("git", "clone", "https://github.com/JacopoMangiavacchi/SwiftNormalization.git")

Cloning into 'SwiftNormalization'...
remote: Enumerating objects: 176, done.        
remote: Counting objects: 100% (176/176), done.        
remote: Compressing objects: 100% (91/91), done.        
remote: Total 176 (delta 96), reused 140 (delta 66), pack-reused 0        
Receiving objects: 100% (176/176), 23.77 KiB | 5.94 MiB/s, done.
Resolving deltas: 100% (96/96), done.


0


In [0]:
%include "SwiftNormalization/Sources/SwiftNormalization/Helpers.swift"
%include "SwiftNormalization/Sources/SwiftNormalization/Normalizer.swift"
%include "SwiftNormalization/Sources/SwiftNormalization/MinMaxNormalizer.swift"

**Swift** code to use the normalization package

In [0]:
func flattenMatrix<T: BinaryFloatingPoint>(_ matrix : [[T]]) -> [T] {
    let size = matrix.count
    guard size > 0 else { return [] }

    let count = matrix[0].count
    var allEqualCount = true
    for m in 0..<size {
        if matrix[m].count != count {
          allEqualCount = false
          break
        }
    }
  
    guard allEqualCount else { return [] }
  
    let total = count * size
    var vector = [T](repeating: 0, count: total)
    for i in stride(from: 0, to: total, by: size) {
        for m in 0..<size {
            vector[i+m] = matrix[m][i/3]
        }
    }

    return vector
}

In [0]:
var heightNormalizer = MinMaxNormalizer<Float>()
var weightNormalizer = MinMaxNormalizer<Float>()

let (xArray, yVector) = getArrayDataset()

let xNormalizedVector = flattenMatrix([heightNormalizer.normalized(xArray[0]), 
                                       xArray[1], 
                                       xArray[2]])
let yNormalizedVector = weightNormalizer.normalized(yVector)

**Swift TensorFlow** code for Model creation and Training

In [0]:
import TensorFlow

// Not really needed - Just for test custom activation
@differentiable()
func linear(input: Tensor<Float>) -> Tensor<Float> {
    return input
}

struct LinearRegression: Layer {
    var l1: Dense<Float>
    init(variables: Int = 1) {
        l1 = Dense<Float>(inputSize: variables, outputSize: 1, activation: linear) // linear Not really needed
        // l1 = Dense<Float>(inputSize: variables, outputSize: 1)
    }
    
    @differentiable(wrt: (self, input))
    func applied(to input: Tensor<Float>, in context: Context) -> Tensor<Float> {
        return l1.applied(to: input, in: context)
    }
}

In [8]:
let x = Tensor<Float>(shape: [Int32(10000), 3], scalars: xNormalizedVector)
let y = Tensor<Float>(shape: [Int32(10000), 1], scalars: yNormalizedVector)

print(x.shape)
print(y.shape)

TensorShape(dimensions: [10000, 3])
TensorShape(dimensions: [10000, 1])


In [9]:
let optimizer = SGD<LinearRegression, Float>(learningRate: 0.03)
var model = LinearRegression(variables: 3)

let trainingContext = Context(learningPhase: .training)
for epoch in 1...3000 {
    let (cost, 𝛁model) = model.valueWithGradient { m -> Tensor<Float> in
        let ŷ = m.applied(to: x, in: trainingContext)
        return meanSquaredError(predicted: ŷ, expected: y)
    }
    optimizer.update(&model.allDifferentiableVariables, along: 𝛁model)
  
    if epoch % 1000 == 0 {
        print("Epoch: \(epoch) Cost: \(cost)")
    }
}

Epoch: 1000 Cost: 0.002481593
Epoch: 2000 Cost: 0.0024073587
Epoch: 3000 Cost: 0.0023867376


**Swift TensorFlow** code for Inferencing

In [10]:
let height = heightNormalizer.normalize(Float(76.8))
let heightTensor = TensorElementLiteral<Float>(floatLiteral: height)

let normPredictedWeight = model.applied(to: [[heightTensor, 0, 1]])  //Height, Female, Male
let predictedWeight = weightNormalizer.denormalize(Float(normPredictedWeight[0][0])!)

print(predictedWeight)

235.06735
